In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clean_raw_data.csv
/kaggle/input/clean_data_mean.csv
/kaggle/input/raw_data.csv
/kaggle/input/clean_data_GCN.csv
/kaggle/input/FillZero_minmax_baseline/clean_data_minmax_fill-zero.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/val/val_week1_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/test/test_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/clean_data_week2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/FillZero_minmax_baseline/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/val/val_week1_2_3.csv
/kaggle/input/FillZero_minmax_baseline/clean_week3/test/test_week3.csv
/kaggle/input/FillZero_minmax_bas

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score

<ipython-input-2-63192117ceec>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
# Biến global cho base path
BASE_PATH = "/kaggle/input/Median_minmax_baseline_version2"
# Tuần và số phần fold
weeks = ['week1', 'week2', 'week3', 'week4']
fold_parts = 5

# Tạo five_fold_files
five_fold_files = {
    week: [
        f"{BASE_PATH}/clean_{week}/train/5-folds/data_part_{i}.csv"
        for i in range(1, fold_parts + 1)
    ]
    for week in weeks
}

# Tạo file_validation
file_validation = {
    'week1': [f"{BASE_PATH}/clean_week1/val/val_week1.csv"],
    'week2': [f"{BASE_PATH}/clean_week2/val/val_week1_2.csv"],
    'week3': [f"{BASE_PATH}/clean_week3/val/val_week1_2_3.csv"],
    'week4': [f"{BASE_PATH}/clean_week4/val/val_week1_2_3_4.csv"]
}

# Tạo file_test
file_test = {
    week: [f"{BASE_PATH}/clean_{week}/test/test_{week}.csv"]
    for week in weeks
}


In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
   # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    

    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)

    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)

    # Xây dựng mô hình 4-layer stacked LSTM
    def build_model(hp):
        model = tf.keras.Sequential()
        
        # LSTM layer 1
        model.add(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train_res.shape[1], 1)
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 2
        model.add(layers.LSTM(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 3
        model.add(layers.LSTM(
            units=hp.Int('units_3', min_value=32, max_value=256, step=32),
            return_sequences=True
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))
        
        # LSTM layer 4
        model.add(layers.LSTM(
            units=hp.Int('units_4', min_value=32, max_value=256, step=32),
            return_sequences=False
        ))
        model.add(layers.Dropout(rate=hp.Float('dropout_4', min_value=0.1, max_value=0.5, step=0.1)))

        # Lớp đầu ra
        model.add(layers.Dense(5, activation='softmax'))

        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        return model

    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'lstm_tuning_week{week_number}'
    )

    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)

    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params


In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    week: f"{BASE_PATH}/clean_{week}/train/clean_data_{week}.csv"
    for week in weeks
}

# Định nghĩa file_validation theo quy luật riêng
file_validation = {
    f"week{idx + 1}": f"{BASE_PATH}/clean_week{idx + 1}/val/val_week{'_'.join(str(i) for i in range(1, idx + 2))}.csv"
    for idx in range(len(weeks))
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validation["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validation["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validation["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validation["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 10 Complete [00h 05m 11s]
val_accuracy: 0.9987797141075134

Best val_accuracy So Far: 0.9993898868560791
Total elapsed time: 00h 52m 11s
Best Parameters for Week 1:
units_1: 64
dropout_1: 0.1
units_2: 32
dropout_2: 0.2
units_3: 128
dropout_3: 0.30000000000000004
units_4: 224
dropout_4: 0.4
learning_rate: 0.005906654353027764

Best Parameters for Week 2:
units_1: 64
dropout_1: 0.30000000000000004
units_2: 96
dropout_2: 0.4
units_3: 160
dropout_3: 0.5
units_4: 256
dropout_4: 0.1
learning_rate: 0.0032414632197183516

Best Parameters for Week 3:
units_1: 192
dropout_1: 0.5
units_2: 224
dropout_2: 0.4
units_3: 128
dropout_3: 0.1
units_4: 32
dropout_4: 0.5
learning_rate: 0.0027547362874968547

Best Parameters for Week 4:
units_1: 192
dropout_1: 0.1
units_2: 256
dropout_2: 0.2
units_3: 160
dropout_3: 0.5
units_4: 96
dropout_4: 0.5
learning_rate: 0.0009833354050242252


In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "auc_roc_per_label": [],    # AUC từng lớp
        "auc_roc_macro": [],        # AUC macro
        "auc_roc_weighted": [],     # AUC weighted (tự tính)
        "precision_macro": [],
        "recall_macro": [],
        "f1_macro": [],
        "precision_weighted": [],
        "recall_weighted": [],
        "f1_weighted": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        model = build_lstm_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        # Tính AUC-ROC
        try:
            # Tính AUC macro và theo từng lớp với OvR
            auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
            auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
            # Tính AUC weighted: tính trọng số theo số mẫu của từng lớp
            supports = np.bincount(y_test_classes, minlength=5)
            auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
        except Exception as e:
            print(f"Lỗi khi tính AUC: {e}")
            auc_macro = np.nan
            auc_per_class = [np.nan] * 5
            auc_weighted = np.nan
            
        # Lưu kết quả của fold hiện tại
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["auc_roc_per_label"].append(auc_per_class)  # AUC từng lớp
        week_results["auc_roc_macro"].append(auc_macro)          # AUC macro
        week_results["auc_roc_weighted"].append(auc_weighted)      # AUC weighted
        week_results["confusion_matrices"].append(conf_matrix)
        week_results["precision_macro"].append(precision_macro)
        week_results["recall_macro"].append(recall_macro)
        week_results["f1_macro"].append(f1_macro)
        week_results["precision_weighted"].append(precision_weighted)
        week_results["recall_weighted"].append(recall_weighted)
        week_results["f1_weighted"].append(f1_weighted)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.nanmean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.nanmean(week_results["f1_score_per_label"], axis=0)
    average_auc_per_label = np.nanmean(week_results["auc_roc_per_label"], axis=0)
    average_confusion_matrix = np.nanmean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    average_accuracy = np.nanmean(week_results["accuracy_per_fold"])
    average_precision_macro = np.nanmean(week_results["precision_macro"])
    average_recall_macro = np.nanmean(week_results["recall_macro"])
    average_f1_macro = np.nanmean(week_results["f1_macro"])
    average_auc_macro = np.nanmean(week_results["auc_roc_macro"])
    average_precision_weighted = np.nanmean(week_results["precision_weighted"])
    average_recall_weighted = np.nanmean(week_results["recall_weighted"])
    average_f1_weighted = np.nanmean(week_results["f1_weighted"])
    average_auc_weighted = np.nanmean(week_results["auc_roc_weighted"])


    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label,
        "Average AUC": average_auc_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    # In kết quả Accuracy và Macro metrics
    print("\n=== Average Accuracy ===")
    print(f"{average_accuracy:.4f}")
    print("\n=== Average Macro Metrics ===")
    print(f"Macro Precision: {average_precision_macro:.4f}")
    print(f"Macro Recall: {average_recall_macro:.4f}")
    print(f"Macro F1-Score: {average_f1_macro:.4f}")
    print(f"Macro AUC-ROC: {average_auc_macro:.4f}")
    print("\n=== Average Weighted Metrics ===")
    print(f"Weighted Precision: {average_precision_weighted:.4f}")
    print(f"Weighted Recall: {average_recall_weighted:.4f}")
    print(f"Weighted F1-Score: {average_f1_weighted:.4f}")
    print(f"Weighted AUC-ROC: {average_auc_weighted:.4f}")
    print("\n=== Average Metrics per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)
    
    # Cập nhật kết quả cho tuần hiện tại
    week_results.update({
        "average_accuracy": average_accuracy,
        "average_precision_macro": average_precision_macro,
        "average_recall_macro": average_recall_macro,
        "average_f1_macro": average_f1_macro,
        "average_auc_macro": average_auc_macro,
        "average_precision_weighted": average_precision_weighted,
        "average_recall_weighted": average_recall_weighted,
        "average_f1_weighted": average_f1_weighted,
        "average_auc_weighted": average_auc_weighted,
        "average_metrics_df": metrics_df,
        "average_confusion_matrix": confusion_df,
        "average_train_times": average_train_time,
        "average_test_times": average_test_time,
    })
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 64, 'dropout_1': 0.1, 'units_2': 32, 'dropout_2': 0.2, 'units_3': 128, 'dropout_3': 0.30000000000000004, 'units_4': 224, 'dropout_4': 0.4, 'learning_rate': 0.005906654353027764}
Fold 1: Using file /kaggle/input/Median_minmax_baseline_version2/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.5958 - loss: 0.1472 - val_accuracy: 0.6176 - val_loss: 0.1259
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6242 - loss: 0.1300 - val_accuracy: 0.6645 - val_loss: 0.1323
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6439 - loss: 0.1239 - val_accuracy: 0.6535 - val_loss: 0.1316
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6524 - loss: 0.1245 - val_accuracy: 0.6432 - val_loss: 0.1279
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6461 - loss: 0.1239 - val_accuracy: 0.6576 - val_loss: 0.1261
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6630 - loss: 0.1222 - val_accuracy: 0.6710 - val_loss: 0.1208
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6723 - loss: 0.1214 - val_accuracy: 0.6809 - val_loss: 0.1151
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6513 - loss: 0.1222 - val_accuracy: 0.6786 -

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn

Fold 2: Using file /kaggle/input/Median_minmax_baseline_version2/clean_week1/train/5-folds/data_part_2.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.5974 - loss: 0.1509 - val_accuracy: 0.6256 - val_loss: 0.1302
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6149 - loss: 0.1395 - val_accuracy: 0.6123 - val_loss: 0.1305
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6210 - loss: 0.1327 - val_accuracy: 0.6264 - val_loss: 0.1283
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6195 - loss: 0.1281 - val_accuracy: 0.6123 - val_loss: 0.1260
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6202 - loss: 0.1281 - val_accuracy: 0.6721 - val_loss: 0.1201
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.6353 - loss: 0.1242 - val_accuracy: 0.6710 - val_loss: 0.1210
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6268 - loss: 0.1292 - val_accuracy: 0.702

...

Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.5998 - loss: 0.1538 - val_accuracy: 0.7076 - val_loss: 0.1084
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7119 - loss: 0.1082 - val_accuracy: 0.7705 - val_loss: 0.0915
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8047 - loss: 0.0841 - val_accuracy: 0.8772 - val_loss: 0.0557
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8501 - loss: 0.0687 - val_accuracy: 0.8795 - val_loss: 0.0508
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8677 - loss: 0.0609 - val_accuracy: 0.8891 - val_loss: 0.0445
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8941 - loss: 0.0462 - val_accuracy: 0.9119 - val_loss: 0.0377
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9008 - loss: 0.0424 - val_accuracy: 0.9245 - val_loss: 0.0301
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9137 - loss: 0.0372 - val_accura

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.6176 - loss: 0.1554 - val_accuracy: 0.7490 - val_loss: 0.1096
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7431 - loss: 0.1035 - val_accuracy: 0.7925 - val_loss: 0.0897
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8288 - loss: 0.0749 - val_accuracy: 0.8642 - val_loss: 0.0613
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8662 - loss: 0.0584 - val_accuracy: 0.9104 - val_loss: 0.0380
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8925 - loss: 0.0492 - val_accuracy: 0.9035 - val_loss: 0.0412
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8972 - loss: 0.0452 - val_accuracy: 0.9058 - val_loss: 0.0349
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9075 - loss: 0.0390 - val_accuracy: 0.9348 - val_loss: 0.0266
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9153 - loss: 0.0358 - val_accuracy

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.6129 - loss: 0.1550 - val_accuracy: 0.7117 - val_loss: 0.1070
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7303 - loss: 0.1053 - val_accuracy: 0.8539 - val_loss: 0.0678
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8260 - loss: 0.0747 - val_accuracy: 0.8616 - val_loss: 0.0564
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8584 - loss: 0.0598 - val_accuracy: 0.9001 - val_loss: 0.0439
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8773 - loss: 0.0524 - val_accuracy: 0.9039 - val_loss: 0.0414
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8883 - loss: 0.0460 - val_accuracy: 0.9329 - val_loss: 0.0305
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9011 - loss: 0.0424 - val_accuracy: 0.9272 - val_loss: 0.0307
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9088 - loss: 0.0380 - val_accuracy: 0

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_accuracy = np.mean(week_results["accuracy_per_fold"])
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Accurancy: {average_accuracy}")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print(f"Average AUC Macro: {average_auc_macro}")
    print(f"Average AUC Weighted: {average_auc_weighted}")
    print("\nAverage Precision, Recall, F1-Score, AUC-ROC per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Accurancy: 0.956684410572052
Average Train Time: 163.7178 seconds
Average Test Time: 0.9104 seconds
Average AUC Macro: 0.9937902589239338
Average AUC Weighted: 0.9970806727787555

Average Precision, Recall, F1-Score, AUC-ROC per Label:
   Label  Average Precision  Average Recall  Average F1-Score  Average AUC
0      0           0.559701        0.658000          0.602116     0.815179
1      1           0.000000        0.000000          0.000000     0.783254
2      2           0.617564        0.377251          0.453388     0.850517
3      3           0.520596        0.478415          0.473553     0.875637
4      4           0.856450        0.871669          0.862387     0.898790

Average Confusion Matrix:
       0    1     2     3       4
0  394.8  0.0  21.4  42.0   141.8
1   57.8  0.0   1.6   9.0    19.0
2   47.2  0.0  62.0  11.0    44.2
3   47.2  0.0   5.2  80.0    34.8
4  165.8  0.0  15.6  24.4  1397.8

=== Results for week2 ===
Average Accurancy: 0.

## Kiểm tra trên tập test

In [11]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

def build_lstm_model(params):
    model = tf.keras.Sequential()
    
    # LSTM layer 1
    model.add(layers.LSTM(
        units=params.get('units_1', 32),
        return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
    )
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))

    # LSTM layer 2
    model.add(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))

    # LSTM layer 3
    model.add(layers.LSTM(
        units=params.get('units_3', 32),
        return_sequences=True
    ))
    model.add(layers.Dropout(rate=params.get('dropout_3', 0.2)))

    # LSTM layer 4
    model.add(layers.LSTM(
        units=params.get('units_4', 32),
        return_sequences=False
    ))
    model.add(layers.Dropout(rate=params.get('dropout_4', 0.2)))

    # Output layer
    model.add(layers.Dense(5, activation='softmax'))  # 5 classes in output

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
        loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy']
    )
    
    return model

In [12]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[f"week{week_num}"].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"{BASE_PATH}/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"{BASE_PATH}/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    model = build_lstm_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average='weighted')
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    
    # Tính AUC-ROC (với one-vs-rest)
    try:
        auc_macro = roc_auc_score(y_test, y_pred, multi_class="ovr", average="macro")
        auc_per_class = roc_auc_score(y_test, y_pred, multi_class="ovr", average=None)
        # Tính AUC weighted tự tính theo trọng số mẫu của từng lớp
        supports = np.bincount(y_test_classes, minlength=5)
        auc_weighted = np.sum(auc_per_class * supports) / np.sum(supports)
    except Exception as e:
        print(f"Lỗi khi tính AUC: {e}")
        auc_macro = np.nan
        auc_per_class = [np.nan] * 5
        auc_weighted = np.nan

    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "accuracy": accuracy,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_weighted": precision_weighted,
        "recall_weighted": recall_weighted,
        "f1_weighted": f1_weighted,
        "auc_macro": auc_macro,
        "auc_weighted": auc_weighted,
        "auc_per_class": auc_per_class,
        "confusion_matrix": conf_matrix
    })
    
    # In kết quả chi tiết
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))

    print("\n=== Macro Averages & Accuracy ===")
    print(f"Macro Precision: {precision_macro:.4f}")
    print(f"Macro Recall: {recall_macro:.4f}")
    print(f"Macro F1-Score: {f1_macro:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\n=== Weighted Averages ===")
    print(f"Weighted Precision: {precision_weighted:.4f}")
    print(f"Weighted Recall: {recall_weighted:.4f}")
    print(f"Weighted F1-Score: {f1_weighted:.4f}")
    
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {auc_macro:.4f}")
    print(f"AUC Weighted: {auc_weighted:.4f}")
    print(f"AUC per Label: {auc_per_class}")
    
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [13]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.3994 - loss: 0.2146 - val_accuracy: 0.6288 - val_loss: 0.2219
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.4547 - loss: 0.2009 - val_accuracy: 0.7286 - val_loss: 0.2249
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4571 - loss: 0.2006 - val_accuracy: 0.4235 - val_loss: 0.3438
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.4799 - loss: 0.1946 - val_accuracy: 0.5001 - val_loss: 0.2445
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.4719 - loss: 0.1947 - val_accuracy: 0.0000e+00 - val_loss: 0.3469
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.4521 - loss: 0.2020 - val_accuracy: 0.4580 - val_loss: 0.2894
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.4951 - loss: 0.1967 - val_accuracy: 0.6383 - val_loss: 0.2382
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5032 - loss: 0.1937 

In [14]:
process_week(2, best_params, results)


=== Processing Week 2 ===


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.4100 - loss: 0.2118 - val_accuracy: 0.4640 - val_loss: 0.2514
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.4842 - loss: 0.1888 - val_accuracy: 0.5640 - val_loss: 0.2611
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.5108 - loss: 0.1817 - val_accuracy: 0.6433 - val_loss: 0.2304
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5325 - loss: 0.1761 - val_accuracy: 0.7398 - val_loss: 0.2223
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5409 - loss: 0.1696 - val_accuracy: 0.5328 - val_loss: 0.2809
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5512 - loss: 0.1672 - val_accuracy: 0.7521 - val_loss: 0.1837
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5699 - loss: 0.1603 - val_accuracy: 0.6468 - val_loss: 0.2147
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5848 - loss:

In [15]:
process_week(3, best_params, results)


=== Processing Week 3 ===


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.4222 - loss: 0.2110 - val_accuracy: 0.3959 - val_loss: 0.3634
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.4621 - loss: 0.1982 - val_accuracy: 0.5954 - val_loss: 0.2566
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.4938 - loss: 0.1885 - val_accuracy: 0.7304 - val_loss: 0.2017
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5105 - loss: 0.1838 - val_accuracy: 0.7324 - val_loss: 0.2470
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5245 - loss: 0.1784 - val_accuracy: 0.7042 - val_loss: 0.2173
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5402 - loss: 0.1759 - val_accuracy: 0.7131 - val_loss: 0.2256
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5542 - loss: 0.1689 - val_accuracy: 0.6964 - val_loss: 0.2151
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.5574 - loss: 0

In [16]:
process_week(4, best_params, results)


=== Processing Week 4 ===


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - accuracy: 0.5075 - loss: 0.1860 - val_accuracy: 0.7782 - val_loss: 0.1538
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7125 - loss: 0.1082 - val_accuracy: 0.7528 - val_loss: 0.1146
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.7960 - loss: 0.0702 - val_accuracy: 0.7353 - val_loss: 0.1072
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8202 - loss: 0.0585 - val_accuracy: 0.7116 - val_loss: 0.1079
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.8440 - loss: 0.0496 - val_accuracy: 0.7274 - val_loss: 0.1082
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8626 - loss: 0.0431 - val_accuracy: 0.8087 - val_loss: 0.0598
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.8894 - loss: 0.0360 - val_accuracy: 0.9319 - val_loss: 0.0184
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9015 - loss

In [17]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Accurancy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Macro Precision: {result['precision_macro']}")
    print(f"  Macro Recall: {result['recall_macro']}")
    print(f"  Macro F1-Score: {result['f1_macro']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")
    print("\n=== AUC-ROC ===")
    print(f"AUC Macro: {result['auc_macro']:.4f}")
    print(f"AUC Weighted: {result['auc_weighted']:.4f}")
    print(f"AUC per Label: {result['auc_per_class']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 529.62 seconds
  Test Time: 0.85 seconds
  Accurancy: 0.6687004270896888
  Precision: [0.68773234 0.12068966 0.42105263 0.30894309 0.87428571]
  Recall: [0.49333333 0.25454545 0.54368932 0.73076923 0.76347305]
  F1-Score: [0.57453416 0.16374269 0.47457627 0.43428571 0.81513053]
  Macro Precision: 0.48254068649508086
  Macro Recall: 0.5571620785857168
  Macro F1-Score: 0.49245387289174347
  Confusion Matrix:
[[185  60  30  28  72]
 [ 21  14   6   3  11]
 [ 18   3  56  16  10]
 [  6   2   3  76  17]
 [ 39  37  38 123 765]]

=== AUC-ROC ===
AUC Macro: 0.8059
AUC Weighted: 0.7971
AUC per Label: [0.73890717 0.77521809 0.85464389 0.85245866 0.8083754 ]
Week 2:
  Train Time: 520.18 seconds
  Test Time: 0.84 seconds
  Accurancy: 0.5082367297132397
  Precision: [0.86904762 0.1443299  0.21843003 0.21276596 0.86211901]
  Recall: [0.19466667 0.76363636 0.62135922 0.57692308 0.59281437]
  F1-Score: [0.31808279 0.24277457 0.32323232 0.3108